# Gráficos de Cleveland: CO Congruentes vs Incongruentes

Este notebook genera **gráficos de Cleveland** para visualizar el Cambio de Opinión (CO) comparando respuestas **congruentes** e **incongruentes** con la ideología del participante.

## Concepto:

Para cada ítem mostramos:
- **Punto azul (●)**: CO cuando el ítem es **Congruente** con la ideología
- **Punto rojo (●)**: CO cuando el ítem es **Incongruente** con la ideología
- **Línea conectando**: Muestra la diferencia entre ambas condiciones
- **Color de línea**:
  - Verde: Mayor CO en Incongruentes (>0.05)
  - Gris: Diferencia pequeña (±0.05)
  - Naranja: Mayor CO en Congruentes (<-0.05)

## Interpretación:

- **Valores más altos**: Mayor cambio de opinión (más frecuente)
- **Línea hacia la derecha**: Más cambio cuando el ítem es incongruente
- **Línea hacia la izquierda**: Más cambio cuando el ítem es congruente
- **Hipótesis**: Se espera mayor CO en ítems incongruentes (conflicto ideológico)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Ruta al archivo
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Elecciones = os.path.join(Ruta_Base, 'df_Elecciones.xlsx')

# Cargar datos
df_Elecciones = pd.read_excel(Archivo_Elecciones)

print(f'✓ Datos cargados:')
print(f'  - {len(df_Elecciones)} registros')
print(f'  - {len(df_Elecciones.columns)} columnas')

# Verificar columnas CO
columnas_co_cong = [col for col in df_Elecciones.columns if 'CO_Item_' in col and '_Congruente' in col]
columnas_co_incong = [col for col in df_Elecciones.columns if 'CO_Item_' in col and '_Incongruente' in col]

print(f'\n  - Variables CO_Congruente: {len(columnas_co_cong)}')
print(f'  - Variables CO_Incongruente: {len(columnas_co_incong)}')

## 2. Preparar Datos para Gráfico

In [ ]:
# Definir ítems
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

print(f'Total de ítems: {len(Todos_Items)}')
print(f'Ítems: {Todos_Items}')

# Calcular promedios por ítem
datos_grafico = []

for item in Todos_Items:
    var_cong = f'CO_Item_{item}_Congruente'
    var_incong = f'CO_Item_{item}_Incongruente'
    
    if var_cong in df_Elecciones.columns and var_incong in df_Elecciones.columns:
        # Calcular promedios (ignorando NaN)
        media_cong = df_Elecciones[var_cong].mean()
        media_incong = df_Elecciones[var_incong].mean()
        
        # Calcular diferencia (Incongruente - Congruente)
        diferencia = media_incong - media_cong
        
        # Determinar tipo de ítem
        tipo = 'Progresista' if item in Items_Progresistas else 'Conservador'
        
        datos_grafico.append({
            'Item': item,
            'Tipo': tipo,
            'CO_Congruente': media_cong,
            'CO_Incongruente': media_incong,
            'Diferencia': diferencia
        })

# Crear DataFrame
df_grafico = pd.DataFrame(datos_grafico)

print(f'\n✓ Datos preparados para {len(df_grafico)} ítems')
print(f'\nPrimeras filas:')
print(df_grafico.head(10))

## 3. Función para Crear Gráfico de Cleveland

In [ ]:
def Crear_Grafico_Cleveland_CO(df, titulo, nombre_archivo=None, carpeta_destino='Graficos_Cleveland'):
    """
    Crea un gráfico de Cleveland mostrando CO Congruente vs Incongruente.
    
    Parámetros:
    -----------
    df : DataFrame
        DataFrame con columnas: Item, CO_Congruente, CO_Incongruente, Diferencia
    titulo : str
        Título del gráfico
    nombre_archivo : str, optional
        Nombre del archivo (sin extensión)
    carpeta_destino : str
        Carpeta donde guardar
    """
    
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Ordenar por diferencia (descendente)
    df_sorted = df.sort_values('Diferencia', ascending=True).reset_index(drop=True)
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Parámetros visuales
    y_positions = np.arange(len(df_sorted))
    
    # Dibujar líneas conectando los puntos
    for idx, row in df_sorted.iterrows():
        cong_val = row['CO_Congruente']
        incong_val = row['CO_Incongruente']
        diferencia = row['Diferencia']
        
        # Determinar color de línea según diferencia
        if diferencia > 0.05:  # Más CO en Incongruentes
            color_linea = '#2ecc71'  # Verde
            alpha = 0.6
        elif diferencia < -0.05:  # Más CO en Congruentes
            color_linea = '#e74c3c'  # Rojo/Naranja
            alpha = 0.6
        else:  # Diferencia pequeña
            color_linea = '#95a5a6'  # Gris
            alpha = 0.3
        
        # Dibujar línea
        ax.plot([cong_val, incong_val], [idx, idx], 
                color=color_linea, linewidth=1.5, alpha=alpha, zorder=1)
    
    # Dibujar puntos de Congruente
    ax.scatter(df_sorted['CO_Congruente'], y_positions, 
               s=120, c='#3498db', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Congruente', zorder=3, alpha=0.8)
    
    # Dibujar puntos de Incongruente
    ax.scatter(df_sorted['CO_Incongruente'], y_positions, 
               s=120, c='#e74c3c', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Incongruente', zorder=3, alpha=0.8)
    
    # Configurar ejes
    ax.set_yticks(y_positions)
    ax.set_yticklabels([f'Ítem {int(item)}' for item in df_sorted['Item']], fontsize=10)
    
    ax.set_xlabel('Cambio de Opinión (proporción)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Ítem', fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    
    # Grid
    ax.grid(True, axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Leyenda personalizada
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', 
               markersize=10, label='Congruente', markeredgecolor='white', markeredgewidth=1),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c', 
               markersize=10, label='Incongruente', markeredgecolor='white', markeredgewidth=1),
        Line2D([0], [0], color='#2ecc71', linewidth=2, label='Mayor en Incong (>0.05)'),
        Line2D([0], [0], color='#95a5a6', linewidth=2, label='Similar (±0.05)'),
        Line2D([0], [0], color='#e74c3c', linewidth=2, label='Mayor en Cong (<-0.05)')
    ]
    
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10, 
              framealpha=0.95, edgecolor='gray')
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar
    if nombre_archivo:
        ruta_completa = os.path.join(carpeta_destino, f'{nombre_archivo}.png')
        plt.savefig(ruta_completa, dpi=300, bbox_inches='tight', facecolor='white')
        print(f'✅ Gráfico guardado: {ruta_completa}')
    
    plt.show()
    
    return fig, ax

## 4. Gráfico Principal: Todos los Ítems

In [ ]:
# Crear gráfico principal
fig, ax = Crear_Grafico_Cleveland_CO(
    df_grafico,
    titulo='Cambio de Opinión:\nCongruente vs Incongruente con Ideología',
    nombre_archivo='Cleveland_CO_Congruente_vs_Incongruente'
)

## 5. Análisis Estadístico de Diferencias

In [ ]:
print('='*70)
print('ANÁLISIS: CO CONGRUENTE VS INCONGRUENTE')
print('='*70)

print(f'\n📊 Estadísticas Generales:')
print(f'  Promedio CO_Congruente: {df_grafico["CO_Congruente"].mean():.4f}')
print(f'  Promedio CO_Incongruente: {df_grafico["CO_Incongruente"].mean():.4f}')
print(f'  Diferencia promedio (Incong - Cong): {df_grafico["Diferencia"].mean():.4f}')

print(f'\n📈 Distribución de Diferencias:')
mayor_incong = len(df_grafico[df_grafico['Diferencia'] > 0.05])
similares = len(df_grafico[(df_grafico['Diferencia'] >= -0.05) & (df_grafico['Diferencia'] <= 0.05)])
mayor_cong = len(df_grafico[df_grafico['Diferencia'] < -0.05])

print(f'  Mayor CO en Incongruentes (>0.05): {mayor_incong} ítems ({mayor_incong/len(df_grafico)*100:.1f}%)')
print(f'  Similar (±0.05): {similares} ítems ({similares/len(df_grafico)*100:.1f}%)')
print(f'  Mayor CO en Congruentes (<-0.05): {mayor_cong} ítems ({mayor_cong/len(df_grafico)*100:.1f}%)')

print(f'\n🏆 Top 5 Ítems con MAYOR diferencia (Incong > Cong):')
print('-'*70)
top_incong = df_grafico.nlargest(5, 'Diferencia')[['Item', 'Tipo', 'CO_Congruente', 'CO_Incongruente', 'Diferencia']]
print(top_incong.to_string(index=False))

print(f'\n🏆 Top 5 Ítems con MENOR diferencia (Cong > Incong):')
print('-'*70)
top_cong = df_grafico.nsmallest(5, 'Diferencia')[['Item', 'Tipo', 'CO_Congruente', 'CO_Incongruente', 'Diferencia']]
print(top_cong.to_string(index=False))

print('\n' + '='*70)

## 6. Test Estadístico: ¿Son Diferentes?

In [ ]:
from scipy import stats

print('='*70)
print('TEST ESTADÍSTICO')
print('='*70)

# Test t pareado (porque son los mismos ítems en dos condiciones)
t_stat, p_value = stats.ttest_rel(df_grafico['CO_Incongruente'], df_grafico['CO_Congruente'])

print('\n📊 Test t Pareado:')
print('-'*70)
print(f'  H0: No hay diferencia entre CO_Congruente y CO_Incongruente')
print(f'  Ha: Hay diferencia significativa')
print(f'\n  Estadístico t: {t_stat:.4f}')
print(f'  Valor p: {p_value:.4f}')

if p_value < 0.05:
    print(f'\n  ✅ SIGNIFICATIVO (p < 0.05)')
    if df_grafico['Diferencia'].mean() > 0:
        print(f'     El CO es SIGNIFICATIVAMENTE MAYOR en ítems Incongruentes')
    else:
        print(f'     El CO es SIGNIFICATIVAMENTE MAYOR en ítems Congruentes')
else:
    print(f'\n  ❌ NO SIGNIFICATIVO (p ≥ 0.05)')
    print(f'     No hay diferencia significativa entre condiciones')

# Tamaño del efecto (d de Cohen)
diferencias = df_grafico['CO_Incongruente'] - df_grafico['CO_Congruente']
d_cohen = diferencias.mean() / diferencias.std()

print(f'\n  Tamaño del efecto (d de Cohen): {d_cohen:.4f}')
if abs(d_cohen) < 0.2:
    magnitud = 'PEQUEÑO'
elif abs(d_cohen) < 0.5:
    magnitud = 'MEDIANO'
else:
    magnitud = 'GRANDE'
print(f'  Magnitud: {magnitud}')

print('\n' + '='*70)

## 7. Comparación por Tipo de Ítem

In [ ]:
print('='*70)
print('COMPARACIÓN: ÍTEMS PROGRESISTAS vs CONSERVADORES')
print('='*70)

# Agrupar por tipo
por_tipo = df_grafico.groupby('Tipo').agg({
    'CO_Congruente': ['mean', 'std'],
    'CO_Incongruente': ['mean', 'std'],
    'Diferencia': ['mean', 'std'],
    'Item': 'count'
}).round(4)

print('\n📊 Estadísticas por Tipo:')
print(por_tipo)

print('\n💡 Interpretación:')
df_prog = df_grafico[df_grafico['Tipo'] == 'Progresista']
df_cons = df_grafico[df_grafico['Tipo'] == 'Conservador']

dif_prog = df_prog['Diferencia'].mean()
dif_cons = df_cons['Diferencia'].mean()

print(f'\n  Ítems Progresistas:')
print(f'    - Diferencia promedio: {dif_prog:.4f}')
if dif_prog > 0:
    print(f'    - Más CO cuando el ítem es Incongruente con ideología')
else:
    print(f'    - Más CO cuando el ítem es Congruente con ideología')

print(f'\n  Ítems Conservadores:')
print(f'    - Diferencia promedio: {dif_cons:.4f}')
if dif_cons > 0:
    print(f'    - Más CO cuando el ítem es Incongruente con ideología')
else:
    print(f'    - Más CO cuando el ítem es Congruente con ideología')

print('\n' + '='*70)

## 8. Guardar Tabla Resumen

In [ ]:
# Crear carpeta de salida
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Cleveland')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar tabla de datos
Archivo_Salida = os.path.join(Carpeta_Salida, 'Resumen_CO_Congruente_vs_Incongruente.xlsx')
df_grafico_export = df_grafico.copy()
df_grafico_export.to_excel(Archivo_Salida, index=False)

print(f'✅ Tabla resumen guardada: {Archivo_Salida}')
print(f'   {len(df_grafico_export)} ítems × {len(df_grafico_export.columns)} columnas')

## 9. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: GRÁFICOS DE CLEVELAND - CO CONGRUENTE VS INCONGRUENTE')
print('='*70)

print('\n📊 Análisis completado:')
print(f'  - Ítems analizados: {len(df_grafico)}')
print(f'  - Gráficos generados: 1 (Cleveland plot)')
print(f'  - Comparación: Congruente vs Incongruente')

print('\n📁 Archivos generados:')
print('  - Cleveland_CO_Congruente_vs_Incongruente.png')
print('  - Resumen_CO_Congruente_vs_Incongruente.xlsx')

print('\n🎯 Hallazgos clave:')
print(f'  - {mayor_incong} ítems: Mayor CO en Incongruentes')
print(f'  - {mayor_cong} ítems: Mayor CO en Congruentes')
print(f'  - {similares} ítems: Diferencia mínima')

print('\n💡 Interpretación teórica:')
print('  - ESPERADO: Mayor CO en ítems incongruentes (conflicto ideológico)')
if df_grafico['Diferencia'].mean() > 0:
    print('  - RESULTADO: ✅ Confirmado - Mayor CO en incongruentes')
else:
    print('  - RESULTADO: ❌ No confirmado - Patrón opuesto o nulo')

print('\n' + '='*70)
print('✅ ANÁLISIS COMPLETADO')
print('='*70)